In [24]:
!pip install pytorch_metric_learning
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
from google.colab import drive
import os
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(path)

import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers,regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import math
import json
from sklearn.model_selection import train_test_split
import pandas as pd
import jieba
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score,f1_score
from tensorflow.keras.layers import concatenate
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data
import matplotlib.pyplot as plt
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.utils import common_functions as c_f
from pytorch_metric_learning.utils.inference import InferenceModel, MatchFinder
import operator as opt

torch.set_default_tensor_type(torch.DoubleTensor)

SEED = 30

torch.manual_seed(SEED) 
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True 

embedding_size = 300
sequence_length = 5000
num_classes = 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


RuntimeError: ignored

In [ ]:
f = open("./data/url+text/longtext_train",encoding='utf-8')   
label,text = [],[]
for line in f:
  values = line.split()
  label.append(int(values[0]))
  text.append(values[1:])
print(len(text))
print(text[0])

f = open("./data/url+text/url_train3",encoding='utf-8')   
url = []
for line in f:
  url.append(line)
print(len(url))
print(url[0])

In [ ]:
max_num_words_text = 30000
tokenizer=Tokenizer(num_words=max_num_words_text,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~� ',oov_token='<OOV>')
tokenizer.fit_on_texts(text)
vocab_text={word:i+1 for i, word in enumerate(tokenizer.word_index) if i+1<max_num_words_text}

from gensim.models import Word2Vec
w2vmodel = Word2Vec(sentences=text, size=300, window=5, min_count=1, workers=4)
#w2vmodel.save("w2v_model.model")
#w2vmodel = Word2Vec.load("w2v_model.model")
embedding_matrix_text = np.zeros((len(vocab_text) + 1, 300))
for word, i in vocab_text.items():
    try:
        embedding_vector = w2vmodel.wv[str(word)]
        embedding_matrix_text[i] = embedding_vector
    except KeyError:
        continue
embedding_matrix_text = torch.from_numpy(embedding_matrix_text)
embedding_matrix_text.shape

In [ ]:
max_num_words_url = 10000
tokenizer=Tokenizer(num_words=max_num_words_url,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~� ',oov_token='<OOV>')
tokenizer.fit_on_texts(url)
vocab_url={word:i+1 for i, word in enumerate(tokenizer.word_index) if i+1<max_num_words_url}

urlmodel = Word2Vec(sentences=url, size=300, window=5, min_count=1, workers=4)
embedding_matrix_url = np.zeros((len(vocab_url) + 1, 300))
for word, i in vocab_url.items():
    try:
        embedding_vector = urlmodel.wv[str(word)]
        embedding_matrix_url[i] = embedding_vector
    except KeyError:
        continue
embedding_matrix_url = torch.from_numpy(embedding_matrix_url)
embedding_matrix_url.shape

In [ ]:
embedding_matrix = torch.cat((embedding_matrix_text, embedding_matrix_url), 0)
embedding_matrix.shape

In [ ]:
text_sequence_length, url_sequence_length = 5000, 18

#rootdata = pad_sequences(tokenizer.texts_to_sequences(text), maxlen=text_sequence_length)
rootdata = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url), maxlen=url_sequence_length)+30000],axis=1)
rootdata.shape

In [ ]:
class TextDataset(Data.Dataset):
    """LongText2000 dataset."""

    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels

In [ ]:
torch_data = TextDataset(rootdata, label)

train_size = int(len(torch_data) * 0.6)
validate_size = int(len(torch_data) * 0.2)
test_size = len(torch_data) - train_size - validate_size
train_set, valid_set, test_set = Data.random_split(torch_data, [train_size, validate_size, test_size])


train_loader = Data.DataLoader(train_set, batch_size=32, shuffle=True, drop_last=False, num_workers=4, pin_memory=True)
print(len(train_loader))

In [ ]:
# text和url信息在输入特征层合并
class textCNN(nn.Module):
    def __init__(self):
        super(textCNN, self).__init__()
        Knum = 256 
        Ks = [3,4,5]
        
        self.embed = nn.Embedding(len(vocab_url)+1, embedding_size, _weight=embedding_matrix) 
        
        self.convs = nn.ModuleList([nn.Conv2d(1,Knum,(K,embedding_size)) for K in Ks])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(len(Ks)*Knum,128) 
        
    def forward(self,x):
        x = self.embed(x) #(N,W,D)
        x = x.unsqueeze(1) #(N,Ci,W,D)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs] # len(Ks)*(N,Knum,W)
        x = [F.max_pool1d(line,line.size(2)).squeeze(2) for line in x]  # len(Ks)*(N,Knum)
        
        x = torch.cat(x,1) #(N,Knum*len(Ks))
        
        x = self.dropout(x)
        logit = self.fc(x)
        return logit

if hasattr(torch.cuda, 'empty_cache'):
	  torch.cuda.empty_cache()

In [ ]:
def train(model, loss_func, mining_func, train_loader, train_set, valid_set, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.cuda(), labels.cuda()
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        print(
            "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                epoch, batch_idx, loss, mining_func.num_triplets
            )
        )
    model.eval()
    best_acc = 0
    with torch.no_grad():
        train_embeddings, train_labels = get_all_embeddings(train_set, model)
        valid_embeddings, valid_labels = get_all_embeddings(valid_set, model)
        train_labels = train_labels.squeeze(1)
        valid_labels = valid_labels.squeeze(1)
        train_embeddings = train_embeddings.cpu()
        valid_embeddings = valid_embeddings.cpu()
        train_labels = train_labels.cpu()
        valid_labels = valid_labels.cpu()
        accuracies = accuracy_calculator.get_accuracy(
            valid_embeddings, train_embeddings, valid_labels, train_labels, False
        )
        acc = accuracies["precision_at_1"]
        if best_acc < acc:
            best_acc = acc
            torch.save(model.state_dict(), 'triplet_loss_0426_2.pt')
        print("Valid set accuracy (Precision@1) = {}".format(acc))

In [ ]:
### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

In [ ]:
### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    train_embeddings = train_embeddings.cpu()
    test_embeddings = test_embeddings.cpu()
    train_labels = train_labels.cpu()
    test_labels = test_labels.cpu()

    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

In [ ]:
EPOCH = 5
model = textCNN().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard"
)

accuracy_calculator = AccuracyCalculator(
    include=(
        "precision_at_1",
        "r_precision",
        "mean_average_precision_at_r",
    ),
    k="max_bin_count"
)
### pytorch-metric-learning stuff ###

for epoch in range(EPOCH):
    train(model, loss_func, mining_func, train_loader, train_set, valid_set, optimizer, epoch)

In [ ]:
model = textCNN().cuda()
model.load_state_dict(torch.load('triplet_loss_0426_2.pt'))
test(train_set, test_set, model, accuracy_calculator)

In [ ]:
def cleanurl(line):
    line = delgoogleweblight(line)
    whiteurlparts = ['/', '.', ':']
    comp = []
    for i in list(filter(bool, re.split('([^a-zA-Z])',line))):
        if i in whiteurlparts or len(i)>=2:
          comp.append(i)
    return comp

def delgoogleweblight(url):
    a = url.strip().split('/')
    if a[2]=='googleweblight.com':
        a.remove('')
        a.remove('googleweblight.com')
        a.remove('fp?u='+a[0])
        return '/'.join(a)
    else:
        return url

In [ ]:
df_annotation = pd.read_excel('./data/标注网站（短文本）.xlsx')
url_test = list(df_annotation['url'].apply(cleanurl))
print(url_test[0])

f = open("./data/Annotation_data.txt",encoding='utf-8')   
test_label,test_text = [],[]
cnt = 0
for line in f:
  values = line.split()
  test_text.append(values)
  test_label.append(0)

test_data = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(test_text), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url_test), maxlen=url_sequence_length)+30000],axis=1)
data_test = TextDataset(test_data, test_label)

In [ ]:
def kNN(dataset, labels, testdata, k):
    distSquareMat = (dataset - testdata) ** 2
    distSquareSums = distSquareMat.sum(axis=1) 
    distances = distSquareSums ** 0.5 
    sortedIndices = distances.argsort()  #array.argsort(),默认axis=0从小到大排序，得到排序后的下标位置
    indices = sortedIndices[:k] #取距离最小的k个值对应的小标位置
    p,n = 0,0
    for i in indices:
        label = labels[i]
        if label == 1:
          p += 1
        else:
          n += 1
    if p>n:
      return 1
    elif p<n:
      return 0
    else:
      return '无法区分'

In [ ]:
train_x, train_y = get_all_embeddings(train_set, model)
train_x = train_x.cpu().numpy()
train_y = train_y.cpu().numpy()

In [ ]:
test_x, test_y = get_all_embeddings(data_test, model)
test_x = test_x.cpu().numpy()
test_y = test_y.cpu().numpy()

In [ ]:
for i in test_x:
    result = kNN(train_x, train_y, i, 11)
    print(result)

In [ ]:
"""
计算uncertainty的还有预测的几个函数
"""
import torch
from torch import nn

def normalization(p, n):
    unnormalized_pred = torch.tensor([p, n])
    normalized_pred = nn.Softmax(dim=0)(unnormalized_pred)
    return normalized_pred


def least_confidence(p, n):
    tensor = normalization(p, n)
    p = tensor[0]
    q = tensor[1]
    if p > q:
        return p
    else:
        return q


def entropy_reduction(p, n):
    tensor = normalization(p, n)
    tensor = tensor * torch.log(tensor)
    print("tensor = ", tensor)
    p = tensor[0]
    q = tensor[1]
    return p + q
    #之后取最小的为最需要标注的，故此处为+


def margin_sampling(p, n):
    tensor = normalization(p, n)
    print("tensor = ", tensor)
    p = tensor[0]
    q = tensor[1]
    if p > q:
        return p - q
    else:
        return q - p


def uncertainty(p, n, uncertainty):
    if uncertainty == "least_confidence":
      return least_confidence(p, n)
    elif uncertainty == "entropy_reduction":
      return entropy_reduction(p, n)
    elif uncertainty == "margin_sampling":
      return margin_sampling(p, n)
    else:
      return (p, n)


print(uncertainty(0.0, 11.0, "least_confidence"))


def kNN_uncertainty(dataset, labels, testdata, k):
    distSquareMat = (dataset - testdata) ** 2
    distSquareSums = distSquareMat.sum(axis=1) 
    distances = distSquareSums ** 0.5 
    sortedIndices = distances.argsort()  #array.argsort(),默认axis=0从小到大排序，得到排序后的下标位置
    indices = sortedIndices[:k] #取距离最小的k个值对应的小标位置
    p,n = 0.0,0.0
    for i in indices:
        label = labels[i]
        if label == 1:
          p += 1.0
        else:
          n += 1.0
    return (p, n)


tensor(1.0000)


In [ ]:
"""
进行预测并把最需要标注的数据对应的预测文件中的行数存储到uncertain_list内
"""
f = open("data/url+text/longtext_pred",encoding='utf-8')   
test_label,test_text = [],[]
for line in f:
  values = line.split()
  test_text.append(values[1:])
  test_label.append(0)

f = open("data/url+text/url_pred",encoding='utf-8')   
url_test = []
for line in f:
  line = line.split()
  url_test.append(line)


test_data = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(test_text), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url_test), maxlen=url_sequence_length)],axis=1)
data_test = TextDataset(test_data, test_label)

train_x, train_y = get_all_embeddings(train_set, model)
train_x = train_x.cpu().numpy()
train_y = train_y.cpu().numpy()

test_x, test_y = get_all_embeddings(data_test, model)
test_x = test_x.cpu().numpy()
test_y = test_y.cpu().numpy()


pred_dict = dict()
for line_number, i in enumerate(test_x):
    print(line_number)
    # result = kNN(train_x, train_y, i, 11)
    # print(result)
    result1 = kNN_uncertainty(train_x, train_y, i, 11)
    print("result1 = ", result1)
    result2 = uncertainty(result1[0], result1[1], "least_confidence")
    print("result2 = ", result2)
    pred_dict[line_number] = result2

pred_list = sorted(pred_dict.items(), key=lambda x: x[1], reverse=False)
uncertain_list = pred_list[:3000]
print(uncertain_list)

#uncertain_list is a list of tuple, with the first element represent the line number(begin with 0) in predic file need to be added to the new train file.


100%|██████████| 292/292 [02:23<00:00,  2.03it/s]


Streaming output truncated to the last 5000 lines.
result2 =  tensor(1.0000)
7660
result1 =  (0.0, 11.0)
result2 =  tensor(1.0000)
7661
result1 =  (11.0, 0.0)
result2 =  tensor(1.0000)
7662
result1 =  (11.0, 0.0)
result2 =  tensor(1.0000)
7663
result1 =  (6.0, 5.0)
result2 =  tensor(0.7311)
7664
result1 =  (10.0, 1.0)
result2 =  tensor(0.9999)
7665
result1 =  (2.0, 9.0)
result2 =  tensor(0.9991)
7666
result1 =  (5.0, 6.0)
result2 =  tensor(0.7311)
7667
result1 =  (0.0, 11.0)
result2 =  tensor(1.0000)
7668
result1 =  (0.0, 11.0)
result2 =  tensor(1.0000)
7669
result1 =  (0.0, 11.0)
result2 =  tensor(1.0000)
7670
result1 =  (0.0, 11.0)
result2 =  tensor(1.0000)
7671
result1 =  (0.0, 11.0)
result2 =  tensor(1.0000)
7672
result1 =  (3.0, 8.0)
result2 =  tensor(0.9933)
7673
result1 =  (7.0, 4.0)
result2 =  tensor(0.9526)
7674
result1 =  (4.0, 7.0)
result2 =  tensor(0.9526)
7675
result1 =  (9.0, 2.0)
result2 =  tensor(0.9991)
7676
result1 =  (9.0, 2.0)
result2 =  tensor(0.9991)
7677
result1 

In [ ]:
"""
把新增的数据（uncertain_list中的）放到原训练数据集的最后
"""
f_in = open("data/url+text/longtext_pred",encoding='utf-8')
f_out = open("data/url+text/longtext_train1", 'a')
f_out.write("\n")
for line_number, line in enumerate(f_in):
  select = False
  for uncertain_line in uncertain_list:
    if uncertain_line[0] == line_number:
      select = True
      break
  if select:
    f_out.write(line)
    print("line_number = ", line_number)

f_in.close()
f_out.close()

f_in = open("data/url+text/url_pred",encoding='utf-8')
f_out = open("data/url+text/url_train1", 'a')
f_out.write("\n")
for line_number, line in enumerate(f_in):
  select = False
  for uncertain_line in uncertain_list:
    if uncertain_line[0] == line_number:
      select = True
      break
  if select:
    f_out.write(line)

f_in.close()
f_out.close()

line_number =  0
line_number =  1
line_number =  2
line_number =  3
line_number =  4
line_number =  5
line_number =  6
line_number =  7
line_number =  8
line_number =  10
line_number =  11
line_number =  12
line_number =  14
line_number =  17
line_number =  19
line_number =  20
line_number =  25
line_number =  26
line_number =  27
line_number =  29
line_number =  35
line_number =  36
line_number =  41
line_number =  46
line_number =  47
line_number =  48
line_number =  49
line_number =  50
line_number =  53
line_number =  56
line_number =  58
line_number =  59
line_number =  60
line_number =  61
line_number =  62
line_number =  66
line_number =  68
line_number =  71
line_number =  73
line_number =  74
line_number =  76
line_number =  77
line_number =  80
line_number =  81
line_number =  85
line_number =  88
line_number =  90
line_number =  92
line_number =  93
line_number =  94
line_number =  95
line_number =  96
line_number =  97
line_number =  100
line_number =  102
line_number =  10